# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Import pprint
from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../WeatherPy/output/city_weather_data.csv"
weather_df = pd.read_csv(weather_csv_file)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,port alfred,-33.59,26.89,16.74,48.0,0.0,3.41,ZA,1.601306e+09
1,rikitea,-23.12,-134.97,23.54,81.0,4.0,3.82,PF,1.601306e+09
2,codrington,-38.27,141.97,5.08,93.0,45.0,4.03,AU,1.601306e+09
3,atuona,-9.80,-139.03,25.22,75.0,6.0,7.40,PF,1.601306e+09
4,hobart,-42.88,147.33,9.00,76.0,90.0,1.50,AU,1.601306e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=gkey)

In [17]:
# Create a humidity heatmap layer
fig = gmaps.figure()

# Store Lat and Lng as locations
locations = weather_df[["Lat", "Lng"]].astype(float)

# Store humidity as weight
humidity = weather_df["Humidity"].astype(float)

# Retrieve max weather humidity for max intensity in heat map
max_intensity = max(weather_df["Humidity"])

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=max_intensity,
                                point_radius=2.0)

# Add heat layer on figure
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Find cities with ideal temperature

# Store previously created df into a new df
ideal_weather_df = weather_df

# A max temperature lower than 27 degrees celcius but higher than 21.
ideal_weather_df = ideal_weather_df.loc[(weather_df["Max Temp"] < 27) & (weather_df["Max Temp"] > 21)]

# Wind speed less than 4.5 m/s.
ideal_weather_df = ideal_weather_df.loc[weather_df["Wind Speed"] < 4.5]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[weather_df["Cloudiness"] == 0]

# Drop any null values
ideal_weather_df = ideal_weather_df.dropna()

# Display all cities with ideal weather
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,rodeo,38.03,-122.27,24.44,32.0,0.0,2.07,US,1.601306e+09
100,altar,30.70,-111.82,23.33,61.0,0.0,0.45,MX,1.601306e+09
140,guerrero negro,27.98,-114.06,23.42,66.0,0.0,3.57,MX,1.601306e+09
219,srivardhan,18.03,73.02,26.59,80.0,0.0,0.80,IN,1.601306e+09
271,santa fe,-31.63,-60.70,23.89,28.0,0.0,0.45,AR,1.601306e+09
287,kununurra,-15.77,128.73,25.00,73.0,0.0,3.14,AU,1.601306e+09
549,tucuman,-26.82,-65.22,24.00,31.0,0.0,3.60,AR,1.601306e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named `hotel_df`
hotel_df = ideal_weather_df

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Check df
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,rodeo,38.03,-122.27,24.44,32.0,0.0,2.07,US,1.601306e+09,
100,altar,30.70,-111.82,23.33,61.0,0.0,0.45,MX,1.601306e+09,
140,guerrero negro,27.98,-114.06,23.42,66.0,0.0,3.57,MX,1.601306e+09,
219,srivardhan,18.03,73.02,26.59,80.0,0.0,0.80,IN,1.601306e+09,
271,santa fe,-31.63,-60.70,23.89,28.0,0.0,0.45,AR,1.601306e+09,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": gkey
}

# Use lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # Retrieve lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f'Retrieving Results for Index {index}: {row["City"]}')
    
    # make request and print url
    response = requests.get(base_url, params=params).json()
    
    #pprint(response)
    
    # Append generated hotel data into df with try-except
    try:
        print(f'The closest hotel is {response["results"][0]["name"]}')
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
          
    except:
        print("Missing field/result... skipping.")
          
    print("--------------------")

Retrieving Results for Index 9: rodeo
The closest hotel is Vallejo
--------------------
Retrieving Results for Index 100: altar
The closest hotel is Altar Municipality
--------------------
Retrieving Results for Index 140: guerrero negro
The closest hotel is Guerrero Negro
--------------------
Retrieving Results for Index 219: srivardhan
The closest hotel is Shrivardhan
--------------------
Retrieving Results for Index 271: santa fe
The closest hotel is Santa Fe
--------------------
Retrieving Results for Index 287: kununurra
The closest hotel is Kununurra
--------------------
Retrieving Results for Index 549: tucuman
The closest hotel is San Miguel de Tucumán
--------------------


In [8]:
# Check dataframe's new column
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,rodeo,38.03,-122.27,24.44,32.0,0.0,2.07,US,1.601306e+09,Vallejo
100,altar,30.70,-111.82,23.33,61.0,0.0,0.45,MX,1.601306e+09,Altar Municipality
140,guerrero negro,27.98,-114.06,23.42,66.0,0.0,3.57,MX,1.601306e+09,Guerrero Negro
219,srivardhan,18.03,73.02,26.59,80.0,0.0,0.80,IN,1.601306e+09,Shrivardhan
271,santa fe,-31.63,-60.70,23.89,28.0,0.0,0.45,AR,1.601306e+09,Santa Fe
287,kununurra,-15.77,128.73,25.00,73.0,0.0,3.14,AU,1.601306e+09,Kununurra
549,tucuman,-26.82,-65.22,24.00,31.0,0.0,3.60,AR,1.601306e+09,San Miguel de Tucumán


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Store Lat and Lng as locations
locations = hotel_df[["Lat", "Lng"]]

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
#fig = gmaps.figure(layout={'width': '500px', 'height': '400px'})
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))